# Dataset

Get 10000 images from ccpd_base file, and split it into train set, test set and validation set.(7:2:1)

In [1]:
import os
import random

import shutil
from shutil import copy2
# trainfiles = os.listdir(r"D:\UW\EE567-project\CCPD2019\ccpd_base")  #（图片文件夹）
# # num_train = len(trainfiles)
# num_train = 10000
# print("num_train: " + str(num_train) )
# index_list = list(range(num_train))
# print(index_list)
# random.shuffle(index_list)  # 打乱顺序
# num = 0
# trainDir = r"D:\UW\EE567-project\train"   #（将图片文件夹中的6份放在这个文件夹下）
# validDir = r"D:\UW\EE567-project\val"     #（将图片文件夹中的2份放在这个文件夹下）
# testDir = r"D:\UW\EE567-project\test"   #（将图片文件夹中的2份放在这个文件夹下）
# for i in index_list:
#     fileName = os.path.join(r"D:\UW\EE567-project\CCPD2019\ccpd_base", trainfiles[i])  #（图片文件夹）+图片名=图片地址
#     if num < num_train*0.7:  # 7:1:2
#         print(str(fileName))
#         copy2(fileName, trainDir)
#     elif num < num_train*0.8:
#         print(str(fileName))
#         copy2(fileName, validDir)
#     else:
#         print(str(fileName))
#         copy2(fileName, testDir)
#     num += 1

Detect license plates from images and save detected license plates into `./dataset`

In [2]:
import cv2
import os
import numpy as np

# 参考 https://blog.csdn.net/qq_36516958/article/details/114274778
# https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#2-create-labels
from PIL import Image

test_data_path = r'D:\UW\EE567-project\test'
train_data_path = r'D:\UW\EE567-project\train'
val_data_path = r'D:\UW\EE567-project\val'

test_path = r"D:\UW\EE567-project\dataset\test"
train_path = r"D:\UW\EE567-project\dataset\train"
val_path = r"D:\UW\EE567-project\dataset\val"

In [3]:


# provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
# alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
#              'X', 'Y', 'Z', 'O']
# ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
#        'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']
# num = 0
# def make_dataset(input_path, output_path):
#     for filename in os.listdir(input_path):
#         global num
#         num += 1
#         result = ""
#         _, _, box, points, plate, brightness, blurriness = filename.split('-')
#         list_plate = plate.split('_')  # 读取车牌
#         result += provinces[int(list_plate[0])]
#         result += alphabets[int(list_plate[1])]
#         result += ads[int(list_plate[2])] + ads[int(list_plate[3])] + ads[int(list_plate[4])] + ads[int(list_plate[5])] + ads[int(list_plate[6])]
#         print(result)
#         img_path = os.path.join(input_path, filename)
#         img = cv2.imread(img_path)
#         assert os.path.exists(img_path), "image file {} dose not exist.".format(img_path)

#         box = box.split('_')  # 车牌边界
#         box = [list(map(int, i.split('&'))) for i in box]

#         xmin = box[0][0]
#         xmax = box[1][0]
#         ymin = box[0][1]
#         ymax = box[1][1]

#         img = Image.fromarray(img)
#         img = img.crop((xmin, ymin, xmax, ymax))  # 裁剪出车牌位置
#         img = img.resize((94, 24), Image.LANCZOS)
#         img = np.asarray(img)  # 转成array,变成24*94*3

#         output_file = os.path.join(output_path, "{}.jpg".format(result))
#         cv2.imencode('.jpg', img)[1].tofile(output_file)
#         # 图片中文名会报错
#         # cv2.imwrite(r"K:\MyProject\datasets\ccpd\new\ccpd_2020\rec_images\train\{}.jpg".format(result), img)  # 改成自己存放的路径
# make_dataset(test_data_path, test_path)
# make_dataset(train_data_path, train_path)
# make_dataset(val_data_path, val_path)
# print("Total number of images:{}".format(num))


In [4]:
import numpy as np
import random
import cv2
import os
from torch.utils.data import Dataset

CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-'
         ]
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}

def list_images(directory):
    """
    替代imutils.paths.list_images函数
    递归查找目录中的所有图像文件
    """
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
    image_paths = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_paths.append(os.path.join(root, file))
    
    return image_paths

class LPRDataLoader(Dataset):
    def __init__(self, img_dir, imgSize, lpr_max_len, PreprocFun=None):
        self.img_dir = img_dir
        self.img_paths = []
        for i in range(len(img_dir)):
            self.img_paths += list_images(img_dir[i])
        random.shuffle(self.img_paths)
        self.img_size = imgSize
        self.lpr_max_len = lpr_max_len
        if PreprocFun is not None:
            self.PreprocFun = PreprocFun
        else:
            self.PreprocFun = self.transform
            
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        filename = self.img_paths[index]
        # Image = cv2.imread(filename)
        image = cv2.imdecode(np.fromfile(filename, dtype=np.uint8), -1)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        height, width, _ = image.shape
        if height != self.img_size[1] or width != self.img_size[0]:
            image = cv2.resize(image, self.img_size)
        image = self.PreprocFun(image)
        
        basename = os.path.basename(filename)
        imgname, suffix = os.path.splitext(basename)
        imgname = imgname.split("-")[0].split("_")[0]
        
        label = list()
        for c in imgname:
            # one_hot_base = np.zeros(len(CHARS))
            # one_hot_base[CHARS_DICT[c]] = 1
            label.append(CHARS_DICT[c])
            
        if len(label) == 8:
            if self.check(label) == False:
                print(imgname)
                assert 0, "Error label ^~^!!!"
                
        return image, label, len(label)
    
    def transform(self, img):
        img = img.astype('float32')
        img -= 127.5
        img *= 0.0078125
        img = np.transpose(img, (2, 0, 1))
        return img
    
    def check(self, label):
        if label[2] != CHARS_DICT['D'] and label[2] != CHARS_DICT['F'] \
                and label[-1] != CHARS_DICT['D'] and label[-1] != CHARS_DICT['F']:
            print("Error label, Please check!")
            return False
        else:
            return True

In [5]:
from IPython.display import display

test_dataset = LPRDataLoader(test_path.split(','), (94, 24), 8)

img, label, length = test_dataset[1000]
image = np.transpose(img, (1, 2, 0)) / 0.0078125 + 127.5
image = image.astype(np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pil_image = Image.fromarray(image)
display(pil_image)
print("Label:", [CHARS[i] for i in label])
print("Length:", length)

Label: ['皖', 'A', '0', 'H', '5', '9', '9']
Length: 7


# LPRNet

In [6]:
import torch
import torch.nn as nn

class small_basic_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_basic_block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)


class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            # 模拟MaxPool3d(1,3,3)的行为，但使用MaxPool2d
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            small_basic_block(ch_in=64, ch_out=128),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            # 模拟MaxPool3d(1,3,3)的行为，使用stride=(2,1,2)
            nn.MaxPool2d(kernel_size=3, stride=(1, 2), padding=1),
            small_basic_block(ch_in=128, ch_out=256),  # 修复：输入通道为128
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 10
            small_basic_block(ch_in=256, ch_out=256),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            # 模拟MaxPool3d的行为
            nn.MaxPool2d(kernel_size=3, stride=(1, 2), padding=1),
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(1, 4), stride=1),  # 修复：输入通道为256
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1),
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),
        )
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            if i in [2, 6, 13, 22]:  
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            # 打印每个特征的形状以便调试
            # print(f"Feature {i} shape before pooling: {f.shape}")
            
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)
            
            # 记录池化后形状
            # print(f"Feature {i} shape after pooling: {f.shape}")
            
            # 确保所有特征具有相同的空间维度 (H,W)
            # 获取所有特征的最小高度和宽度
            if i == 0:
                h, w = f.size(2), f.size(3)
            else:
                h = min(h, f.size(2))
                w = min(w, f.size(3))
            
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean + 1e-6)
            global_context.append(f)
        
        # 在连接前调整所有特征到相同的空间维度
        for i in range(len(global_context)):
            # 使用自适应池化或裁剪确保所有特征具有相同的高度和宽度
            if global_context[i].size(2) != h or global_context[i].size(3) != w:
                # 方法1：使用自适应池化
                global_context[i] = nn.functional.adaptive_avg_pool2d(global_context[i], (h, w))
                
                # 或方法2：裁剪
                # global_context[i] = global_context[i][:, :, :h, :w]
        
        # 现在所有特征应该具有相同的空间维度，可以安全连接
        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)

        return logits

In [7]:
model = LPRNet(lpr_max_len=8, class_num=len(CHARS), dropout_rate=0.5)
print(model)

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=(1, 2), padding=1, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(128, 64, kernel_size=(1, 1), stride

# Training

Use DataLoader to load test and train data

In [8]:
from torch.utils.data import DataLoader

train_dataset = LPRDataLoader(train_path.split(','), (94, 24), 8)
val_dataset = LPRDataLoader(val_path.split(','), (94, 24), 8)

def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for sample in batch:
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        # imgs.append(img)
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.int32)
    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=collate_fn)

In [9]:
from torch import optim
from torch.utils.tensorboard import SummaryWriter


model = LPRNet(lpr_max_len=8, class_num=len(CHARS), dropout_rate=0.5)
epoch = 10

# loss function
ctc_loss = nn.CTCLoss(blank=len(CHARS)-1, reduction='mean')

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# optim.Adam(model.parameters(), lr=1e-3)

# cpu训练还是GPU训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using ', device)
model = model.to(device)

# tensorboard
log_folder = "./logs"
if not os.path.exists(log_folder):
    os.mkdir(log_folder)
writer = SummaryWriter(log_dir=log_folder)

# 模型保存路径
save_folder = './model'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

using  cuda:0


In [10]:
def sparse_tuple_for_ctc(T_length=18, lengths=8):
    # T_length = lpr_max_len
    input_lengths = []
    target_lengths = []

    for ch in lengths:
        input_lengths.append(T_length)
        target_lengths.append(ch)

    return tuple(input_lengths), tuple(target_lengths)

In [11]:
def greedy_decode(preds, pred_char=False):
    last_chars_idx = len(CHARS) - 1

    # 贪婪解码 (greedy decode)
    pred_labels = []
    for i in range(preds.shape[0]):
        pred = preds[i, :, :]  # 第i张图片对应的结果，即维度为(66, 18)的二维数组
        pred_label = []
        for j in range(pred.shape[1]):  # 遍历每一列，找到每一列最大值的索引（index）
            pred_label.append(np.argmax(pred[:, j], axis=0))
        no_repeat_blank_label = []
        pre_c = -1
        for c in pred_label:  # 合并重复的索引值部分，删除空白标签，即为-1的值(dropout repeate label and blank label)
            if (pre_c == c) or (c == last_chars_idx):
                if c == last_chars_idx:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        pred_labels.append(no_repeat_blank_label)

    # 解码成字符串
    if pred_char:
        labels = []
        for label in pred_labels:
            lb = ""
            for i in label:
                lb += CHARS[i]
            labels.append(lb)
        return pred_labels, labels
    else:
        return pred_labels

In [12]:
# 训练和测试
verbose_step = 100
total_train_step = 0  # 记录训练次数
total_val_step = 0   # 记录测试次数
acc = 0
for i in range(epoch):
    print("-----Epoch{}:-----".format(i+1))

    # 训练
    model.train()  # 开启训练模式
    for images, labels, lengths in train_loader:
        # get ctc parameters
        input_lengths, target_lengths = sparse_tuple_for_ctc(lengths=lengths)
        # forward
        images = images.to(device)
        logits = model(images)
        log_probs = logits.permute(2, 0, 1) # for ctc loss: T x N x C
        log_probs = log_probs.log_softmax(2).requires_grad_()
        # backprop
        optimizer.zero_grad()  # 优化器中的参数梯度归零
        loss = ctc_loss(log_probs, labels, input_lengths=input_lengths, target_lengths=target_lengths)
        if loss.item() == np.inf:  # 如果梯度爆炸
            continue
        loss.backward()        # 反向传播，计算梯度
        optimizer.step()       # 优化网络
        # 记录loss
        if total_train_step % verbose_step == 0:  # 每verbose_step个batch，显示信息
            print("Training steps:{}, loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
            writer.add_images("Epoch_{}".format(epoch), images, total_train_step)
        total_train_step += 1

    # 测试
    model.eval()  # 关闭dropout
    total_test_loss = 0
    total_acc_num = 0
    Tp, Tn = 0.0, 0.0
    with torch.no_grad():  # 关闭梯度计算
        for images, labels, lengths in val_loader:
            # labels: 1D -> 2D
            targets = []
            start = 0
            for length in lengths:
                label = labels[start:start+length]
                targets.append(label.tolist())
                start += length
            # forward
            images = images.to(device)
            prebs = model(images)
            prebs = prebs.cpu().detach().numpy()
            # greedy decode
            preb_labels = greedy_decode(prebs)
            # calculate
            for i, label in enumerate(preb_labels):
                if len(label) != len(targets[i]):  # 长度不一致
                    Tn += 1
                    continue
                if targets[i] == label:
                    Tp += 1
                else:
                    Tn += 1
        total_val_acc = float(Tp) / float(Tp + Tn)
        print("Accuracy on validation set: {}".format(total_val_acc))
        writer.add_scalar("val_acc", total_val_acc, total_val_step)
        total_val_step += 1

    # 保存模型参数
    if total_val_acc >= acc:  # 保存最好的
        acc = total_val_acc
        model_path = save_folder + "/demo.pth"
        torch.save(model.state_dict(), model_path)
        print("model saved")

writer.close()

-----Epoch1:-----
Training steps:0, loss:9.965800285339355
Training steps:100, loss:1.7906250953674316
Accuracy on validation set: 0.0020833333333333333
model saved
-----Epoch2:-----
Training steps:200, loss:0.36994829773902893
Accuracy on validation set: 0.4947916666666667
model saved
-----Epoch3:-----
Training steps:300, loss:0.16036243736743927
Accuracy on validation set: 0.259375
-----Epoch4:-----
Training steps:400, loss:0.15923193097114563
Accuracy on validation set: 0.8270833333333333
model saved
-----Epoch5:-----
Training steps:500, loss:0.06860341876745224
Accuracy on validation set: 0.7552083333333334
-----Epoch6:-----
Training steps:600, loss:0.09581591188907623
Accuracy on validation set: 0.775
-----Epoch7:-----
Training steps:700, loss:0.06916922330856323
Accuracy on validation set: 0.85625
model saved
-----Epoch8:-----
Training steps:800, loss:0.04877069219946861
Accuracy on validation set: 0.765625
-----Epoch9:-----
Training steps:900, loss:0.053539469838142395
Accuracy 

In [13]:
%load_ext tensorboard

In [14]:
!tensorboard --logdir=logs


^C


In [ ]:
# path = './model/demo.pth'  # ./weights/LPR_mix.pth ./weights/LPR_demo.pth
# # 使用map_location，防止GPU训练的模型导入到CPU上测试会出错
# dic = torch.load(path, map_location=torch.device("cpu"))
# model.load_state_dict(dic)
# # file_path = '../chinese_license_plate_generator/train/藏00S8DQV_green_car_False.jpg'
# file_path = './dataset/test/皖A0C757.jpg'
# image = cv2.imread(file_path)
# height, width, _ = image.shape
# if width != 94 or height != 24:
#     image = cv2.resize(image, (94, 24))  # 缩放
# # 查看图片
# img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# img.show()

In [31]:
path = './model/demo.pth'
# 使用map_location，防止GPU训练的模型导入到CPU上测试会出错
dic = torch.load(path, map_location=torch.device("cpu"))
model.load_state_dict(dic)
model = model.to("cpu")

file_path = './dataset/test/苏E02L55.jpg'
# 读取图像为NumPy数组
numpy_image = cv2.imread(file_path)
height, width, _ = numpy_image.shape
if width != 94 or height != 24:
    numpy_image = cv2.resize(numpy_image, (94, 24))  # 缩放

# 查看图片
img = Image.fromarray(cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB))
display(img)

# 预处理图像 - 使用NumPy操作
numpy_image = numpy_image.astype('float32')
# (height, width, channel) -> (channel, height, width)
numpy_image = np.transpose(numpy_image, (2, 0, 1))
# 归一化
numpy_image = numpy_image / 255.0
# 改变shape
numpy_image = numpy_image.reshape(1, 3, 24, 94)

# 转换为PyTorch张量
tensor_image = torch.from_numpy(numpy_image)
print(tensor_image.shape)  # 应该输出 torch.Size([1, 3, 24, 94])

# 现在可以将张量传递给模型进行推理
model.eval()
with torch.no_grad():
    output = model(tensor_image)
    output = output.cpu().detach().numpy()
    labels_idx, labels = greedy_decode(output, True)
    print(labels)

torch.Size([1, 3, 24, 94])
['皖闽E0L']
